In [1]:
import logging
from dask.distributed import Client, LocalCluster

In [2]:
dask_logger = logging.getLogger("distributed.utils_perf")
dask_logger.setLevel(logging.ERROR)

In [3]:
dask_cluster = LocalCluster(
    n_workers=20,
    threads_per_worker=1)
dask_client = Client(dask_cluster)

In [4]:
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 31.14 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46181,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 31.14 GiB
Comm: tcp://127.0.0.1:39223,Total threads: 1
Dashboard: http://127.0.0.1:36289/status,Memory: 1.56 GiB
Nanny: tcp://127.0.0.1:44543,


# Cleanly shut down Dask cluster

In [5]:
dask_cluster.close()

2023-06-09 16:53:48,940 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


# Join English and Tibetan dataframes

In [5]:
from cai_common.data import TeiLoader, KangyurLoader

In [6]:
en_df = TeiLoader('kangyur').dataframe
en_df['location'] = en_df.location.fillna(0).astype(int)
en_df = dask_client.persist(en_df)

In [7]:
len(en_df)

In toh101, the current page was 453 which is larger than the end page 452 for volume 48, xml:id=UT22084-048-001-4468


19298

In [8]:
en_df.head()

,filename,tohoku_number,volume_number,location,text
0,058-003_toh154-the_sutra_of_the_questions_of_t...,toh154,58,0,The Translation The Noble Mahāyāna Sūtra Th...
1,058-003_toh154-the_sutra_of_the_questions_of_t...,toh154,58,394,Homage to all buddhas and bodhisattvas! Thu...
2,058-003_toh154-the_sutra_of_the_questions_of_t...,toh154,58,395,This is also what causes the variety of being...
3,058-003_toh154-the_sutra_of_the_questions_of_t...,toh154,58,396,"“Lord of nāgas, this dominion of yours like..."
4,058-003_toh154-the_sutra_of_the_questions_of_t...,toh154,58,397,for nonvirtuous thoughts and mental states to...


In [9]:
bo_df = KangyurLoader().remove_new_lines().dataframe
bo_df['location'] = bo_df.location.astype(int)
bo_df = dask_client.persist(bo_df)

In [10]:
len(bo_df)

65461

In [11]:
bo_df.head()

,filename,volume_number,location,text
0,v076_pages.txt,76,0,༄༅། །མདོ་སྡེ་ཨཱཾཿ་ཞུགས་སོ།།
1,v076_pages.txt,76,1,༄༅༅། །རྒྱ་གར་སྐད་དུ། པུ་ཎྱ་བ་ལ་ཨ་བ་དཱ་ན། བོད་ས...
2,v076_pages.txt,76,2,པར་བླངས་ཏེ་གནས་པར་མ་བྱས་པས་ལུས་ཞིག་སྟེ་ཤི་བའི་...
3,v076_pages.txt,76,3,བའི་ལས་དག་བྱེད་པ་ཡིན། །དམ་པ་མིན་པས་མི་རྟོགས་ཏེ...
4,v076_pages.txt,76,4,ཚེ་དང་ལྡན་པ་མ་འགགས་པ་དང་། ཚེ་དང་ལྡན་པ་ཤཱ་རིའི་...


# Load translation model

In [429]:
# en_df.sample(frac=0.1).compute().sample(frac=0.01).sort_values('tohoku_number')
# en_df[en_df.tohoku_number == 'toh257'].compute().sample(frac=0.05).sort_values('tohoku_number')
en_df[en_df.tohoku_number == 'toh686'].compute().sample(frac=0.02).sort_values('tohoku_number')

,filename,tohoku_number,volume_number,location,text
643,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,93,630,"“Now I will teach the amogha eye, the supr..."
389,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,377,"If she becomes his sister, she will emulate V..."
193,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,181,"If someone is suffering from a toothache, the..."
112,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,100,"he displays the jewel to the sky, the downpou..."
644,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,93,1,
687,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,93,44,"He will be free from all danger and fear, all..."
378,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,366,"You are free, O vidyā holder, from hindrances..."
374,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,362,If he mindfully repeats the mantra of the lor...
251,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,239,"He should burn bdellium, mustard seeds, and g..."
694,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,93,51,“There is also the following practice proce...


In [430]:
volume_number, location = 92, 362

In [431]:
bo_df[(bo_df.volume_number == volume_number) & (bo_df.location == location - 1)].compute().text.values[0]

'རས་གཟིགས་དབང་ཕྱུག་གིས། དོན་ཡོད་ཞགས་པའི་སྙིང་པོའི་གསང་སྔགས་འདི་སྨྲས་སོ། །ཨོཾ་ཨ་མོ་གྷ་པཱ་ཤ་ཧྲྀ་ད་ཡ་པདྨོཥྞཱི་ཥ་ཀུ་མཱ་ར་བེ་ཥ་དྷ་ར། པདྨེ་ཤྭ་ར། ཨཱ་བེ་ཤ་ཡ། སརྦ་ཨ་མོ་གྷ་ཀུ་ལ་ས་མ་ཡ་ཧྲྀ་ད་ཡ། སརྦ་སིདྡྷི་ཨ་མོ་གྷ་པྲ་ཡཙྪ་པདྨ། པདྨ་ཧཱུྃ་ཧཱུྃ་ན་མ་སྟུ་ཏེ་སྭཱ་ཧཱ། འདི་ནི་དོན་ཡོད་ཞགས་པའི་སྙིང་པོ་བཀླགས་པས་གྲུབ་པའི་སྒྲུབ་ཐབས་སོ། །ཨོཾ་ཨ་མོ་གྷ་པདྨོ་ཥྞཱི་ཥེ་ཤྭ་ར། པདྨེ་ཤྭ་ར། ས་མནྟཱ་ཡ་སརྦ་ཏ་ཐཱ་ག་ཏཱ་ནཱཾ། ཨ་མོ་གྷ་པདྨོཥྞཱི་ཥ་ས་མ་ཡ་ཨཱ་ཀརྵ་ཡ། པྲ་བེ་ཤ་ཡ། སརྦ་ཀརྨ་སིདྡྷིམྨེ་པྲ་ཡཙྪ་ཨ་བ་ལོ་ཀི་ཏེ་ཤྭ་ར་ཧཱུཾ་ཧཱུཾ། མ་ཧཱ་པདྨོཥྞཱི་ཥ་བྷུ་ཛེ་སྭཱ་ཧཱ། རིགས་ཀྱི་རྒྱལ་པོའི་གསང་སྔགས་སོ། །ཨོཾ་ཨ་ཧཱ་ཙནྜ་པདྨེ་ཤྭ་ར་བི་བི་དྷ་རཱུ་པ། བི་ཀ་ཊ། པདྨ་དཾཥྜ་ཀེ་རཱ་ལ་བྷཱི་ཥ་ཎ། བཱཀྟྲ་སརྦ་དུཥྚ་ཧྲྀ་ད་ཡཾ། ཁཱ་ད་ཡ། བིགྷ་ནཱཾ། པདྨ་དྷྲི་ཀ །ཙི་ཊི། དྲི་ཊི་ཧཱུཾ་སྭཱ་ཧཱ། ཁྲོ་བོའི་རྒྱལ་པོའི་རིགས་ཀྱི་སྙིང་པོའོ། །ནང་གི་དཀྱིལ་འཁོར་གསང་བའི་ལས་རྣམས་སོ། །ཨོཾ་ཨ་མོ་གྷ་པདྨོཥྞཱི་ཥ་པདྨ་པཱ་ཤ་ཀྲོ་དྷ་ཀ །མོ་ཡ་ཀརྨ་པྲ་བེ་ཤ་ཡ་མ་ཧཱ་བཱ་ཤུ་པ་ཏི། ཡ་མ། བ་རུ་ཎ། ཀུ་བེ་ར། བྲཧྨ་བེ་ཥ་དྷ་ར། མ་ཧཱ་ཙཎྜ་བེ་ག །ཨ་མོ་གྷ་ཀུ་ལས་མ་ཡ། པདྨེ་ཧཱུཾ་ཧཱུཾ། ཁྲོ་བོའི་རྒྱལ་པོ་དོན་ཡོད་ཞགས་པའི་པདྨ་གཙུག་ཏོར་གྱི་སྙིང་པ

In [432]:
bo_df[(bo_df.volume_number == volume_number) & (bo_df.location == location)].compute().text.values[0]

'པའི་དཔལ་ཐོབ་བོ། །རིགས་ཀྱི་རྒྱལ་པོའི་གསང་སྔགས་རྒྱས་པར་གྱུར་པ་པདྨ་གཙུག་ཏོར་གྱི་དཀྱིལ་འཁོར་དུ་རིགས་ཀྱི་རྒྱལ་པོ་ལེགས་པར་གྲུབ་པར་འགྱུར་ཏེ། ཐམས་ཅད་དུ་རིགས་ཀྱི་རྒྱལ་པོའི་དངོས་གྲུབ་བཟང་པོ་རྣམས་འོང་བར་འགྱུར་རོ། །རིགས་ཀྱི་རྒྱལ་པོའི་གསང་སྔགས་ཀྱི་རྒྱལ་པོ་དང་། རིག་སྔགས་ཀྱི་རྒྱལ་པོ་མཆོག་གི་དངོས་གྲུབ་ཐམས་ཅད་རབ་ཏུ་སྦྱིན་པར་བྱེད་དོ། །པདྨའི་ཕྱག་གིས་རྗེས་སུ་འཇུག་ཅིང་། དོན་ཡོད་པའི་རིགས་ཀྱི་དངོས་གྲུབ་ཇི་ལྟ་བ་བཞིན་དུ་འགྲུབ་སྟེ། བཀླགས་པས་གྲུབ་པ་འདི་ལ་ཐེ་ཚོམ་མེད་དོ། །པདྨ་གཙུག་ཏོར་གྱི་རིགས་ཀྱི་དངོས་གྲུབ་དང་། པདྨ་ཡངས་པའི་དཀྱིལ་འཁོར་རམ། དོན་ཡོད་ཞགས་པའི་དཀྱིལ་འཁོར་དུ་བཀླགས་ནས་བཟླས་པ་ཙམ་གྱིས་འགྲུབ་སྟེ། རིགས་ཀྱི་རྒྱལ་པོ་དང་། རིག་སྔགས་ཀྱི་རྒྱལ་པོ་གང་ཡིན་པ་དེ་ཐམས་ཅད་དུ་དབང་ཕྱུག་ཏུ་འགྱུར་རོ། །དཀྱིལ་འཁོར་རྣམས་ཀྱི་ཆོ་ག་ཞིབ་མོ་དང་། ཕྱག་རྒྱས་བྱང་ཆུབ་སེམས་དཔའ་ཐམས་ཅད་ཀྱི་རིགས་ཐུན་མོང་བ་ཐམས་ཅད་དུ་རྒྱལ་པོའི་བྱ་བ་བྱེད་པར་འགྱུར་རོ། །སངས་རྒྱས་ཀྱི་ཞིང་སྟོང་ཕྲག་རྣམས་དང་། སངས་རྒྱས་ཀྱི་ཞིང་བརྒྱ་ཕྲག་རྣམས་དང་། གང་གཱའི་ཀླུང་གི་བྱེ་མ་དང་མཉམ་པའི་ཞིང་དུ་རྒྱལ་པོར་འགྱུར་རོ། །རིགས་ཀྱི་རིག་པས་ལྷའི་དབང་ཕྱུག་དང་། ལྷའི་ཚོགས་རྣམས་ཀྱི་དབང་ཕྱུག་ཏུ

In [433]:
bo_df[(bo_df.volume_number == volume_number) & (bo_df.location == location + 1)].compute().text.values[0]

'དང་། སྲིན་པོ་དང་། འབྱུང་པོ་དང་། གཟའ་དང་། བརྗེད་བྱེད་དང་། མི་བཟད་པ་དེ་རྣམས་ཐམས་ཅད་ཁྲོ་བོའི་རྒྱལ་པོ་དྲན་པ་ཙམ་གྱིས་འཇིགས་པར་འགྱུར་རོ། །ཁྲོ་བོའི་རྒྱལ་པོའི་གསང་བ་ཐམས་ཅད་ཀྱི་རིགས་ཀྱི་དམ་ཚིག་ལ་གནས་པ་དང་། དཀྱིལ་འཁོར་དུ་ཕྱག་རྒྱ་གནས་པ་དང་། གསང་སྔགས་ཀྱི་ལྷ་གཞན་གང་ཡིན་པ་རྣམས་ཀྱིས་ཁྲོ་བོའི་རྒྱལ་པོ་དྲན་པ་ཙམ་གྱིས་དེ་རྣམས་ཐམས་ཅད་རིག་སྔགས་འཆང་ལ་སྐྱབས་སུ་འགྲོ་བར་བྱེད་དོ། །དེ་རྣམས་ཐམས་ཅད་རྗེས་སུ་འབྲང་བར་འགྱུར་ཞིང་གནས་པར་བྱེད་དོ། །འགའ་ཞིག་གང་དུ་འགྲོ་བའམ་འདུག་པ་དེ་ལ་དབང་པོ་དང་། སྐབས་གསུམ་པའི་ལྷ་དང་། ཚངས་པ་དང་། ཁྱབ་འཇུག་དང་། དབང་ཕྱུག་ཆེན་པོ་དང་། གཤིན་རྗེ་དང་། ཆུ་ལྷ་དང་། ཀླུའི་རྒྱལ་པོ་རྒྱ་མཚོ་དང་། དགའ་བོ་དང་། ཉེ་དགའ་བོ་དང་། གཞོན་ནུ་དང་། སྡེ་དཔོན་དང་། དེ་བཞིན་དུ་རྒྱལ་པོ་ཆེན་པོ་བཞི་དུས་རྟག་ཏུ་རིག་སྔགས་འཆང་གི་རྗེས་སུ་འབྲང་བར་བྱེད་དོ། །ཐེ་ཚོམ་མེད་དོ། །ཁྲོས་པའི་ལས་རྣམས་སམ། དེ་བཞིན་དུ་དྲག་པོའི་ལས་རྣམས་ཐམས་ཅད་ཀྱང་ཧཱུཾ་ཡིག་ཙམ་གྱིས་དེ་ལ་དངོས་གྲུབ་མཐོང་ངོ་། །ཕྱིར་རྒོལ་བ་དང་། དགྲ་དང་། སེམས་ཅན་མ་རུངས་པ་ཤིན་ཏུ་མི་བཟད་པ་རྣམས་དང་། ཕ་རོལ་གྱི་དམག་ཚོགས་དང་འཇིགས་པ་དང་། རྣམ་པར་འགྲས་པའི་སྐྱེ་བོ་དང་། སྡིག་པ་དེ་རྣམས་ཐམས་ཅད་ཧཱུཾ་ཡ

In [434]:
en_df[(en_df.volume_number == volume_number) & (en_df.location == location)].compute()

,filename,tohoku_number,volume_number,location,text
374,092-001_toh686-the_sovereign_ritual_of_amoghap...,toh686,92,362,If he mindfully repeats the mantra of the lor...


In [435]:
en_text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

In [475]:
en_text = en_df[(en_df.volume_number == volume_number) & (en_df.location == location)].compute().text.values[0]
en_text

' If he mindfully repeats the mantra of the lord of the family, he will obtain the auspicious accomplishment of the lord of the family in the maṇḍala of Padmoṣṇīṣa. Family lords, as well as the mantra and vidyā kings who rule over the family lords’ auspicious accomplishments, will arrive and grant the vidyā holder every supreme accomplishment. Followed by Padmapāṇi, he will win the accomplishments of the amogha family. These kings of mantras will be accomplished through recitation; there is no doubt about this.  “Accomplishment in the Padmoṣṇīṣa family belongs to the great lotus maṇḍala, the maṇḍala of Amoghapāśa, where one becomes accomplished through recitation. Through mere recitation, the vidyā holder will become the lord of the family and the king of the vidyādharas—a sovereign lord in every respect. Using the rites and mudrās of the corresponding maṇḍalas, he will join the same family as the bodhisattvas and will perform the function of a king in every respect. Using the family v

In [19]:
import os

from cai_garland.utils.translator import Translator

In [20]:
translator = Translator(os.path.join("olive-cormorant-nllb", "base-600M"))

translator.num_beams = 20
translator.decoding_length = 200

# translator.prepare_context_encoder("facebook/bart-base")

translator.cuda()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'NllbTokenizer'. 
The class this function is called from is 'CAINllbTokenizerFast'.


In [477]:
bo_text = "།རིགས་ཀྱི་བུ་ཁྱོད་ཀྱང་ད་ལྟར་ཏིང་ངེ་འཛིན་ལས་ལངས་པས་རིགས་ཀྱི་བུ་ཁྱོད་ད་མི་མཇེད་ཀྱི་འཇིག་རྟེན་གྱི་ཁམས་སུ་འགྲོ་བར་བྱའོ།"
bo_text = "།རིགས་ཀྱི་བུ་སངས་རྒྱས་ཀྱི་ཞིང་མི་མཇེད་དེ་ན་ནི་སེམས་ཅན་རྣམས་ཤིན་ཏུ་སྲེད་པའི་ཀུན་ཏུ་སྦྱོར་བ་དང་ལྡན་ཞིང་སྲིད་པའི་ལྟུང་ལ་བཏགས་པ་དག་ཡིན་ཏེ།"
bo_text = "སངས་རྒྱས་ཀྱི་ཞིང་དེ་ན་ནང་ན་ཡང་བུད་མེད་གཟུགས་མི་སྡུག་པ་དང་། ཚུལ་དུ་མི་ཤིས་པ་དང་། མཆོག་ཏུ་གཟུགས་ཀྱི་རྒྱགས་པས་དྲེགས་པ་དང་།"
bo_text = "དྲི་ང་བ་དང་། མི་གཙང་བ་དང་། གཙང་སྦྲར་སྤྱོད་པའི་རྒྱགས་པས་དྲེགས་པ་དང་།"
bo_text = "ངོ་གཉིས་བྱེད་པ་དང་། བདེན་པ་མ་ཡིན་པའི་ཚིག་སྨྲ་བ་དང་། ཚངས་པར་སྤྱོད་པར་ང་རྒྱལ་བྱེད་པ་དང་།"
bo_text = "།བུད་མེད་ཀྱི་ཉེས་པས་དགེ་བའི་རྩ་བ་ཐམས་ཅད་ལས་ཡོངས་སུ་ཉམས་པ་དག་དང་། ངན་སོང་བ་དག་དང་། སེམས་ཅན་དམྱལ་བ་པ་དག་ཏུ་འགྱུར་བ་ཡིན་ནོ།"
bo_text = "།རིགས་ཀྱི་བུ་ཁྱེད་དེ་བཞིན་གཤེགས་པ་ཤཱཀྱ་ཐུབ་པ་དེ་ལ་ངའི་བཀས་ཞུ་བའི་ཕྱིར་ངའི་བཀས་མི་མཇེད་ཀྱི་འཇིག་རྟེན་གྱི་ཁམས་སུ་འགྲོ་བར་སྤྲོ་ན། དེར་ཉི་མའི་སྙིང་པོའི་ཆོས་ཀྱིས་གནས་པ་ཀླུའི་ཡུལ་རྣམ་པར་འཇིག་པར་མཛད་པའི་འོད་ཟེར་གྱི་ལེའུ་སེམས་ཅན་རྣམས་ཀྱི་ལས་ཟད་པར་འགྱུར་བའི་གཟུངས་སྔགས་ལ་ང་དད་པ་སྦྱིན་པར་མཛད་དོ།"

bo_text = "།རིགས་ཀྱི་རྒྱལ་པོའི་གསང་སྔགས་རྒྱས་པར་གྱུར་པ་པདྨ་གཙུག་ཏོར་གྱི་དཀྱིལ་འཁོར་དུ་རིགས་ཀྱི་རྒྱལ་པོ་ལེགས་པར་གྲུབ་པར་འགྱུར་ཏེ།"
bo_text = "སངས་རྒྱས་ཀྱི་ཞིང་བརྒྱ་ཕྲག་རྣམས་དང་། གང་གཱའི་ཀླུང་གི་བྱེ་མ་དང་མཉམ་པའི་ཞིང་དུ་རྒྱལ་པོར་འགྱུར་རོ།"
# bo_text = "།དེ་ནས་དོན་ཡོད་པ་པདྨའི་གཙུག་ཏོར་དྲན་པ་ཙམ་གྱིས། བྱ་བ་ཐམས་ཅད་ནི་གཉིས་སུ་འགྱུར་ཏེ།"
bo_text = "གཞོན་ནུ་དང་། སྡེ་དཔོན་དང་། དེ་བཞིན་དུ་རྒྱལ་པོ་ཆེན་པོ་བཞི་དུས་རྟག་ཏུ་རིག་སྔགས་འཆང་གི་རྗེས་སུ་འབྲང་བར་བྱེད་དོ།"

In [478]:
bo_text, translator.translate(bo_text)

('གཞོན་ནུ་དང་། སྡེ་དཔོན་དང་། དེ་བཞིན་དུ་རྒྱལ་པོ་ཆེན་པོ་བཞི་དུས་རྟག་ཏུ་རིག་སྔགས་འཆང་གི་རྗེས་སུ་འབྲང་བར་བྱེད་དོ།',
 'the prince, the general, and the four great kings will always follow in the footsteps of the vidyadharas.')

In [473]:
# en_text = 'This maṇḍala brings every accomplishment and causes the vidyā holder’s realization, merit, and splendor to grow. It is effective for removing all diseases and eradicating evil and vice. It heals wounds, injuries, and broken bones and destroys poisons and kākhordas. It also removes chronic diseases, ailments, and fevers.'
# en_text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat."

# en_text = 'This maṇḍala brings every accomplishment and causes the vidyā holder’s realization, merit, and splendor to grow. It is effective for removing all diseases and eradicating evil and vice.'
# en_text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco."

# en_text = 'it is the accomplishment of removing all illness. it destroys bad karma and bad karma.'
# en_text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt."

In [391]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [468]:
from torch.nn.functional import log_softmax

def get_total_logits(bo_text, cur_en_text):
    bo_tokens = translator.tokenizer(bo_text, return_tensors="pt")
    with translator.tokenizer.as_target_tokenizer():
        en_tokens = translator.tokenizer(cur_en_text, return_tensors="pt")

    model_inputs = {
        'input_ids': bo_tokens.input_ids.to(translator.model.device),
        'attention_mask': bo_tokens.attention_mask.to(translator.model.device),
        # 'decoder_input_ids': en_tokens.input_ids.to(translator.model.device),
        # 'decoder_attention_mask': en_tokens.attention_mask.to(translator.model.device),
        'labels': en_tokens.input_ids.to(translator.model.device),
    }

    return -float(translator.model(**model_inputs).loss.to("cpu"))# / en_tokens.input_ids.shape[-1]

    model_logits = translator.model(**model_inputs).logits[0].to("cpu")
    # total_logit = float(log_softmax(model_logits[1:][:,en_tokens.input_ids[0][:-1].to("cpu")].diagonal()).sum()) / model_logits.shape[0]
    total_logit = float(log_softmax(model_logits[:,en_tokens.input_ids[0].to("cpu")].diagonal()).sum()) / model_logits.shape[0]
    return(total_logit)

In [479]:
from tqdm.auto import tqdm

all_logits = []
splits = [t.strip() for t in remove_accents(en_text.lower()).replace("!", ".").replace("“", "").replace("—", ", ").split('.')]
for start_idx in tqdm(range(len(splits))):
    for end_idx in tqdm(range(start_idx + 1, len(splits) + 1), leave=False):
        cur_en_text = '. '.join(splits[start_idx:end_idx]) + '.'
        all_logits.append((cur_en_text, get_total_logits(bo_text, cur_en_text)))
# max(all_logits, key=lambda x: x[1])

sorted(all_logits, key=lambda x: -x[1])[:10]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[('there is no doubt about this.', -2.411327600479126),
 ('through mere recitation, the vidya holder will become the lord of the family and the king of the vidyadharas, a sovereign lord in every respect.',
  -2.4114913940429688),
 ('he will also become the king of the devas and their heavenly hosts,  as well as of yaksas, raksasas, and gandharvas.',
  -2.509705066680908),
 ('if he mindfully repeats the mantra of the lord of the family, he will obtain the auspicious accomplishment of the lord of the family in the mandala of padmosnisa.',
  -2.628347396850586),
 ('using the family vidya, he will become king in hundreds of thousands of buddha fields, equal in number to the grains of sand in the ganga river. he will also become the king of the devas and their heavenly hosts,  as well as of yaksas, raksasas, and gandharvas.',
  -2.6626992225646973),
 ('through mere recitation, the vidya holder will become the lord of the family and the king of the vidyadharas, a sovereign lord in every resp